In [1]:
import os
import sys
os.chdir("/home/pamba/Desktop/HomeLearning/ApacheSpark/Python")
os.curdir



if 'SPARK_HOME' not in os.environ:
    os.environ['SPARK_HOME'] = '/home/pamba/spark-2.4.4-bin-hadoop2.7/spark-2.4.4-bin-hadoop2.7'
    

os.environ['JAVA_HOME'] = '/home/pamba/opt/jdk1.8.0_241'
    
# Create a variable for our root path
SPARK_HOME = os.environ['SPARK_HOME']


sys.path.insert(0,os.path.join(SPARK_HOME, "python"))
sys.path.insert(0,os.path.join(SPARK_HOME, "python","lib"))
sys.path.insert(0,os.path.join(SPARK_HOME, "python","lib","pyspark.zip"))
sys.path.insert(0,os.path.join(SPARK_HOME, "python","lib","py4j-0.10.7-src.zip"))


from pyspark import SparkContext
from pyspark import SparkConf


conf=SparkConf()
conf.set("spark.executor.memory", "1g")
conf.set("spark.cores.max", "2")

conf.setAppName("V2 Maestros")


sc = SparkContext('local', conf=conf)

lines=sc.textFile("auto-data.csv")
lines.count()




198

In [2]:
import os
os.chdir("/home/pamba/Desktop/HomeLearning/ApacheSpark/Python")
os.curdir

# Loading data from files

# Load the file. Lazy initialiation

autoData = sc.textFile("auto-data.csv")
autoData.cache()
#LOads only now.



for line in autoData.collect():
    print(line)
    





MAKE,FUELTYPE,ASPIRE,DOORS,BODY,DRIVE,CYLINDERS,HP,RPM,MPG-CITY,MPG-HWY,PRICE
subaru,gas,std,two,hatchback,fwd,four,69,4900,31,36,5118
chevrolet,gas,std,two,hatchback,fwd,three,48,5100,47,53,5151
mazda,gas,std,two,hatchback,fwd,four,68,5000,30,31,5195
toyota,gas,std,two,hatchback,fwd,four,62,4800,35,39,5348
mitsubishi,gas,std,two,hatchback,fwd,four,68,5500,37,41,5389
honda,gas,std,two,hatchback,fwd,four,60,5500,38,42,5399
nissan,gas,std,two,sedan,fwd,four,69,5200,31,37,5499
dodge,gas,std,two,hatchback,fwd,four,68,5500,37,41,5572
plymouth,gas,std,two,hatchback,fwd,four,68,5500,37,41,5572
mazda,gas,std,two,hatchback,fwd,four,68,5000,31,38,6095
mitsubishi,gas,std,two,hatchback,fwd,four,68,5500,31,38,6189
dodge,gas,std,four,hatchback,fwd,four,68,5500,31,38,6229
plymouth,gas,std,four,hatchback,fwd,four,68,5500,31,38,6229
chevrolet,gas,std,two,hatchback,fwd,four,70,5400,38,43,6295
toyota,gas,std,two,hatchback,fwd,four,62,4800,31,38,6338
dodge,gas,std,two,hatchback,fwd,four,68,5500,31,38,6377

In [3]:

autoData.take(5)

['MAKE,FUELTYPE,ASPIRE,DOORS,BODY,DRIVE,CYLINDERS,HP,RPM,MPG-CITY,MPG-HWY,PRICE',
 'subaru,gas,std,two,hatchback,fwd,four,69,4900,31,36,5118',
 'chevrolet,gas,std,two,hatchback,fwd,three,48,5100,47,53,5151',
 'mazda,gas,std,two,hatchback,fwd,four,68,5000,30,31,5195',
 'toyota,gas,std,two,hatchback,fwd,four,62,4800,35,39,5348']

In [12]:

autoData.first()

'MAKE,FUELTYPE,ASPIRE,DOORS,BODY,DRIVE,CYLINDERS,HP,RPM,MPG-CITY,MPG-HWY,PRICE'

In [13]:
autoData.count()

198

In [10]:
# Loading Data From a Collection

collData=sc.parallelize([3,5,4,7,4])
collData.cache()
collData.count()

5

In [5]:

#2. Transformations

#2.1 Map and create a new RDD
tsvData=autoData.map(lambda x: x.replace(",", "\t"))
tsvData.take(5)

['MAKE\tFUELTYPE\tASPIRE\tDOORS\tBODY\tDRIVE\tCYLINDERS\tHP\tRPM\tMPG-CITY\tMPG-HWY\tPRICE',
 'subaru\tgas\tstd\ttwo\thatchback\tfwd\tfour\t69\t4900\t31\t36\t5118',
 'chevrolet\tgas\tstd\ttwo\thatchback\tfwd\tthree\t48\t5100\t47\t53\t5151',
 'mazda\tgas\tstd\ttwo\thatchback\tfwd\tfour\t68\t5000\t30\t31\t5195',
 'toyota\tgas\tstd\ttwo\thatchback\tfwd\tfour\t62\t4800\t35\t39\t5348']

In [6]:
#2.2 Filter and create a new RDD
toyotaData=autoData.filter(lambda x: "toyota" in x)
toyotaData.count()

32

In [7]:
#2.3 FlatMap
words=toyotaData.flatMap(lambda line: line.split(","))
words.take(20)

['toyota',
 'gas',
 'std',
 'two',
 'hatchback',
 'fwd',
 'four',
 '62',
 '4800',
 '35',
 '39',
 '5348',
 'toyota',
 'gas',
 'std',
 'two',
 'hatchback',
 'fwd',
 'four',
 '62']

In [8]:
#2.4 Distinct
for numbData in collData.distinct().collect():
    print(numbData)

3
5
4
7


In [14]:
#2.5 Set operations
words1 = sc.parallelize(["hello","war","peace","world"])
words2 = sc.parallelize(["war","peace","universe"])

for unions in words1.union(words2).distinct().collect():
    print(unions)
    


peace
world
universe
hello
war


In [15]:
for intersects in words1.intersection(words2).collect():
    print(intersects)

peace
war


In [17]:
# 3. Actions
# 3.1 reduce 
collData.reduce(lambda x,y: x+y)


23

In [19]:
# 3.2 find the shortest line
autoData.reduce(lambda x,y: x if len(x) < len(y) else y)

'bmw,gas,std,two,sedan,rwd,six,182,5400,16,22,41315'

In [20]:
# 4. Aggregations
# 4.1 Perform the same work as reduce
seqOp = (lambda x,y: (x+y))
combOp = (lambda x,y: (x+y))
collData.aggregate((0), seqOp, combOp)



23

In [21]:
#Do addition and multiplication at the same time.
#X now becomes a tuple for sequence
seqOp = (lambda x,y: (x[0]+y, x[1]*y))
#both X and Y are tuples
combOp = (lambda x,y: (x[0]+y[0], x[1]*y[1]))
collData.aggregate((0,1), seqOp, combOp)

(23, 1680)

In [22]:
# 5 Functions in spark

# 5.1 cleanse and transform an RDD
def cleanseRDD(autoStr):
    if isinstance(autoStr, int):
        return autoStr
    attList=autoStr.split(",")
    # convert doors to a number
    if attList[3] == "two" :
        attList[3]="2"
    else:
        attList[3]="4"
    # Convert Drive to uppercase
    attList[5] = attList[5].upper()
    return ",".join(attList)

cleanedData=autoData.map(cleanseRDD)
cleanedData.collect()

['MAKE,FUELTYPE,ASPIRE,4,BODY,DRIVE,CYLINDERS,HP,RPM,MPG-CITY,MPG-HWY,PRICE',
 'subaru,gas,std,2,hatchback,FWD,four,69,4900,31,36,5118',
 'chevrolet,gas,std,2,hatchback,FWD,three,48,5100,47,53,5151',
 'mazda,gas,std,2,hatchback,FWD,four,68,5000,30,31,5195',
 'toyota,gas,std,2,hatchback,FWD,four,62,4800,35,39,5348',
 'mitsubishi,gas,std,2,hatchback,FWD,four,68,5500,37,41,5389',
 'honda,gas,std,2,hatchback,FWD,four,60,5500,38,42,5399',
 'nissan,gas,std,2,sedan,FWD,four,69,5200,31,37,5499',
 'dodge,gas,std,2,hatchback,FWD,four,68,5500,37,41,5572',
 'plymouth,gas,std,2,hatchback,FWD,four,68,5500,37,41,5572',
 'mazda,gas,std,2,hatchback,FWD,four,68,5000,31,38,6095',
 'mitsubishi,gas,std,2,hatchback,FWD,four,68,5500,31,38,6189',
 'dodge,gas,std,4,hatchback,FWD,four,68,5500,31,38,6229',
 'plymouth,gas,std,4,hatchback,FWD,four,68,5500,31,38,6229',
 'chevrolet,gas,std,2,hatchback,FWD,four,70,5400,38,43,6295',
 'toyota,gas,std,2,hatchback,FWD,four,62,4800,31,38,6338',
 'dodge,gas,std,2,hatchback

In [23]:
def getMPG(autoStr):
    if isinstance(autoStr, int):
        return autoStr
    attList=autoStr.split(",")
    if attList[9].isdigit():
        return int(attList[9])
    else:
        return 0
    
# 5.2 find average MPG-City for all cars
autoData.reduce(lambda x,y: getMPG(x) + getMPG(y)) \
    / (autoData.count()-1)



25.15228426395939

In [24]:
# 6. Working with key/value RDDs

# 6.1 Create a KV RDD of auto Brand and Horsepower
cylData = autoData.map(lambda x: (x.split(",")[0], \
                                 x.split(",")[7]))

cylData.keys().collect()

['MAKE',
 'subaru',
 'chevrolet',
 'mazda',
 'toyota',
 'mitsubishi',
 'honda',
 'nissan',
 'dodge',
 'plymouth',
 'mazda',
 'mitsubishi',
 'dodge',
 'plymouth',
 'chevrolet',
 'toyota',
 'dodge',
 'honda',
 'toyota',
 'honda',
 'chevrolet',
 'nissan',
 'mitsubishi',
 'dodge',
 'plymouth',
 'mazda',
 'isuzu',
 'mazda',
 'nissan',
 'honda',
 'toyota',
 'toyota',
 'mitsubishi',
 'subaru',
 'nissan',
 'subaru',
 'honda',
 'toyota',
 'honda',
 'honda',
 'nissan',
 'nissan',
 'mazda',
 'subaru',
 'nissan',
 'subaru',
 'dodge',
 'plymouth',
 'mitsubishi',
 'toyota',
 'subaru',
 'volkswagen',
 'toyota',
 'nissan',
 'honda',
 'toyota',
 'toyota',
 'dodge',
 'plymouth',
 'volkswagen',
 'volkswagen',
 'nissan',
 'subaru',
 'toyota',
 'mitsubishi',
 'volkswagen',
 'toyota',
 'nissan',
 'toyota',
 'toyota',
 'mazda',
 'volkswagen',
 'mitsubishi',
 'toyota',
 'honda',
 'mazda',
 'dodge',
 'plymouth',
 'toyota',
 'nissan',
 'honda',
 'subaru',
 'toyota',
 'mitsubishi',
 'mitsubishi',
 'toyota',
 'vo

In [25]:
cylData.take(5)

[('MAKE', 'HP'),
 ('subaru', '69'),
 ('chevrolet', '48'),
 ('mazda', '68'),
 ('toyota', '62')]

In [27]:
# Remove header row
header = cylData.first()
cylHPData = cylData.filter(lambda line: line != header)

In [28]:
# Add a count 1 to each record and then reduce to find totals
brandValues = cylHPData.mapValues(lambda x: (x, 1)) \
    .reduceByKey(lambda x,y: (int(x[0]) + int(y[0]), \
                             x[1] + y[1]))
brandValues.collect()

[('subaru', (1035, 12)),
 ('chevrolet', (188, 3)),
 ('mazda', (1390, 16)),
 ('toyota', (2969, 32)),
 ('mitsubishi', (1353, 13)),
 ('honda', (1043, 13)),
 ('nissan', (1846, 18)),
 ('dodge', (675, 8)),
 ('plymouth', (607, 7)),
 ('isuzu', (168, 2)),
 ('volkswagen', (973, 12)),
 ('saab', (760, 6)),
 ('peugot', (1098, 11)),
 ('volvo', (1408, 11)),
 ('alfa-romero', (376, 3)),
 ('audi', (687, 6)),
 ('bmw', (1111, 8)),
 ('mercury', ('175', 1)),
 ('porsche', (764, 4)),
 ('mercedes-benz', (1170, 8)),
 ('jaguar', (614, 3))]

In [29]:
# Find average by dividing HP total by count total
brandValues.mapValues(lambda x: int(x[0])/int(x[1])).\
     collect()

[('subaru', 86.25),
 ('chevrolet', 62.666666666666664),
 ('mazda', 86.875),
 ('toyota', 92.78125),
 ('mitsubishi', 104.07692307692308),
 ('honda', 80.23076923076923),
 ('nissan', 102.55555555555556),
 ('dodge', 84.375),
 ('plymouth', 86.71428571428571),
 ('isuzu', 84.0),
 ('volkswagen', 81.08333333333333),
 ('saab', 126.66666666666667),
 ('peugot', 99.81818181818181),
 ('volvo', 128.0),
 ('alfa-romero', 125.33333333333333),
 ('audi', 114.5),
 ('bmw', 138.875),
 ('mercury', 175.0),
 ('porsche', 191.0),
 ('mercedes-benz', 146.25),
 ('jaguar', 204.66666666666666)]

In [30]:
# Advanced Spark: Accumulators & Broadcast variables

# function that splits the line as well as counts sedans and
# Speed optimization

# Initialize accumulator
sedanCount = sc.accumulator(0)
hatchbackCount = sc.accumulator(0)

# Set Broadcast variable
sedanText = sc.broadcast("sedan")
hatchbackText = sc.broadcast("hatchback")

def splitLines(line):
    
    global sedanCount
    global hatchbackCount
    
    # Use broadcast variable to do comparison and set accumulator
    if sedanText.value in line:
        sedanCount += 1
    if hatchbackText.value in line:
        hatchbackCount += 1
        
    return line.split(",")

# do the map
splitData = autoData.map(splitLines)
splitData.count()


198

In [31]:
print(sedanCount, hatchbackCount)

92 67


In [32]:
# Advanced Spark : Partitions
collData.getNumPartitions()


1

In [33]:
# Specify no. of partitions.
collData = sc.parallelize([3,5,4,7,4],2)
collData.cache()
collData.count()

5

In [34]:
collData.getNumPartitions()

2

In [2]:
#Code Samples : Spark SQL

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)


In [3]:
# Working with Data Frames

# Create a data frame from a JSON file
empDf = sqlContext.read.json("customerData.json")
empDf.show()

+---+------+------+-----------------+------+
|age|deptid|gender|             name|salary|
+---+------+------+-----------------+------+
| 32|   100|  male|Benjamin Garrison|  3000|
| 40|   200|  male|    Holland Drake|  4500|
| 26|   100|  male|  Burks Velasquez|  2700|
| 51|   100|female|    June Rutledge|  4300|
| 44|   200|  male|    Nielsen Knapp|  6500|
+---+------+------+-----------------+------+



In [4]:
empDf.printSchema()

root
 |-- age: string (nullable = true)
 |-- deptid: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- name: string (nullable = true)
 |-- salary: string (nullable = true)



In [5]:
# Do SQL queries
empDf.select("name").show()

+-----------------+
|             name|
+-----------------+
|Benjamin Garrison|
|    Holland Drake|
|  Burks Velasquez|
|    June Rutledge|
|    Nielsen Knapp|
+-----------------+



In [6]:
empDf.filter(empDf["age"] == 40).show()

+---+------+------+-------------+------+
|age|deptid|gender|         name|salary|
+---+------+------+-------------+------+
| 40|   200|  male|Holland Drake|  4500|
+---+------+------+-------------+------+



In [7]:
empDf.groupBy("gender").count().show()

+------+-----+
|gender|count|
+------+-----+
|female|    1|
|  male|    4|
+------+-----+



In [8]:
empDf.groupBy("deptid").\
    agg({"salary":"avg", "age":"max"}).show()

+------+------------------+--------+
|deptid|       avg(salary)|max(age)|
+------+------------------+--------+
|   200|            5500.0|      44|
|   100|3333.3333333333335|      51|
+------+------------------+--------+



In [9]:
# Create a data frame from a list
deptList = [{'name':'Sales', 'id':'100'},
           {'name':'Engineering', 'id':'200'}]
deptDf = sqlContext.createDataFrame(deptList)
deptDf.show()

/home/pamba/spark-2.4.4-bin-hadoop2.7/spark-2.4.4-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/sql/session.py:346: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead


+---+-----------+
| id|       name|
+---+-----------+
|100|      Sales|
|200|Engineering|
+---+-----------+



In [10]:
# Join the data frames
empDf.join(deptDf, empDf.deptid == deptDf.id).show()

+---+------+------+-----------------+------+---+-----------+
|age|deptid|gender|             name|salary| id|       name|
+---+------+------+-----------------+------+---+-----------+
| 51|   100|female|    June Rutledge|  4300|100|      Sales|
| 26|   100|  male|  Burks Velasquez|  2700|100|      Sales|
| 32|   100|  male|Benjamin Garrison|  3000|100|      Sales|
| 44|   200|  male|    Nielsen Knapp|  6500|200|Engineering|
| 40|   200|  male|    Holland Drake|  4500|200|Engineering|
+---+------+------+-----------------+------+---+-----------+



In [12]:
# Cascading operations
empDf.filter(empDF["age"] > 30).join(deptDf, empDf.deptid == deptDf.id).\
        groupBy("deptid").\
        agg({'salary':'avg', 'age':'max'}).show()
        

NameError: name 'empDF' is not defined

In [13]:
# Register a data frame as table and run SQL statements against
empDf.registerTempTable("employees")

In [14]:
sqlContext.sql("select * from employees where salary > 4000").show()

+---+------+------+-------------+------+
|age|deptid|gender|         name|salary|
+---+------+------+-------------+------+
| 40|   200|  male|Holland Drake|  4500|
| 51|   100|female|June Rutledge|  4300|
| 44|   200|  male|Nielsen Knapp|  6500|
+---+------+------+-------------+------+



In [15]:
#To pandas data frame
empPands = empDf.toPandas()
for index, row in empPands.iterrows():
    print(row["salary"])

3000
4500
2700
4300
6500


In [40]:
String url = "jdbc:postgresql://localhost/test";
Properties props = new Properties();
props.setProperty("user","fred");
props.setProperty("password","secret");
props.setProperty("ssl","true");
Connection conn = DriverManager.getConnection(url, props);

String url = "jdbc:postgresql://localhost/test?user=fred&password=secret&ssl=true";
Connection conn = DriverManager.getConnection(url);

import psycopg2
# Working with Databases

# Make sure that the spark classpaths are set appropriately in the
# spark-defaults.conf file to include the driver files
conn = psycopg2.connect(database="mydb", user="postgres", password="1234", host="127.0.0.1", port="5432")

demoDf = sqlContext.read.format("jdbc").options(
     #url = "jdbc:postgresql://localhost/mydb",
     conn).options(dbtable = "COMPANY").load()

demoDf.show()

TypeError: options() takes 1 positional argument but 2 were given

In [16]:
# Creating data frames from RDD

from pyspark.sql import Row
lines = sc.textFile("auto-data.csv")
# remove the first line
datalines = lines.filter(lambda x: "FUELTYPE" not in x)
datalines.count()

197

In [17]:
parts = datalines.map(lambda l: l.split(","))
autoMap = parts.map(lambda p: Row(make=p[0], \
         body=p[4], hp=int(p[7])))
autoMap.collect()


[Row(body='hatchback', hp=69, make='subaru'),
 Row(body='hatchback', hp=48, make='chevrolet'),
 Row(body='hatchback', hp=68, make='mazda'),
 Row(body='hatchback', hp=62, make='toyota'),
 Row(body='hatchback', hp=68, make='mitsubishi'),
 Row(body='hatchback', hp=60, make='honda'),
 Row(body='sedan', hp=69, make='nissan'),
 Row(body='hatchback', hp=68, make='dodge'),
 Row(body='hatchback', hp=68, make='plymouth'),
 Row(body='hatchback', hp=68, make='mazda'),
 Row(body='hatchback', hp=68, make='mitsubishi'),
 Row(body='hatchback', hp=68, make='dodge'),
 Row(body='hatchback', hp=68, make='plymouth'),
 Row(body='hatchback', hp=70, make='chevrolet'),
 Row(body='hatchback', hp=62, make='toyota'),
 Row(body='hatchback', hp=68, make='dodge'),
 Row(body='hatchback', hp=58, make='honda'),
 Row(body='hatchback', hp=62, make='toyota'),
 Row(body='hatchback', hp=76, make='honda'),
 Row(body='sedan', hp=70, make='chevrolet'),
 Row(body='sedan', hp=69, make='nissan'),
 Row(body='hatchback', hp=68, mak

In [18]:
# Infer the schema, and register the DataFrame as a table
autoDf = sqlContext.createDataFrame(autoMap)
autoDf.registerTempTable("autos")
sqlContext.sql("select * from autos where hp > 200").show()

+-----------+---+-------+
|       body| hp|   make|
+-----------+---+-------+
|    hardtop|207|porsche|
|    hardtop|207|porsche|
|      sedan|262| jaguar|
|convertible|207|porsche|
+-----------+---+-------+



In [2]:
import os
import sys
os.chdir("/home/pamba/Desktop/HomeLearning/ApacheSpark/Python")
os.curdir



if 'SPARK_HOME' not in os.environ:
    os.environ['SPARK_HOME'] = '/home/pamba/spark-2.4.4-bin-hadoop2.7/spark-2.4.4-bin-hadoop2.7'
    

os.environ['JAVA_HOME'] = '/home/pamba/opt/jdk1.8.0_241'
    
# Create a variable for our root path
SPARK_HOME = os.environ['SPARK_HOME']


sys.path.insert(0,os.path.join(SPARK_HOME, "python"))
sys.path.insert(0,os.path.join(SPARK_HOME, "python","lib"))
sys.path.insert(0,os.path.join(SPARK_HOME, "python","lib","pyspark.zip"))
sys.path.insert(0,os.path.join(SPARK_HOME, "python","lib","py4j-0.10.7-src.zip"))


from pyspark import SparkContext
from pyspark import SparkConf

# Configure Sparl settings
conf=SparkConf()
conf.set("spark.executor.memory", "1g")
conf.set("spark.cores.max", "2")

conf.setAppName("V2 Maestros")

# Initialize SparkContext. Run only once. Otherwise you get 
# Context Error
# For streaming create a spark context with 2 threads.
sc = SparkContext('local[2]', conf=conf)

from pyspark.streaming import StreamingContext

In [1]:
# Streaming with simple data

vc = [[-0.1, -0.2], [0.1,0.3], [1.1,1.5], [0.9,0.9]]
dvc = [sc.parallelize(i,1) for i in vc]
ssc = StreamingContext(sc, 2)
input_stream = ssc.queueStream(dvc)

def get_output(rdd):
    print(rdd.collect())
input_stream.foreachRDD(get_output)
ssc.start()
ssc.stop

In [3]:
# Streaming with TCP/IP data

streamContext = StreamingContext(sc,3)

totalLines = 0
lines = streamContext.socketTextStream("localhost", 9000)

# Count lines
totalLines = 0
linesCount = 0

# Word count within RDD
words = lines.flatMap(lambda line: line.split(","))
pairs = words.map(lambda word: (word, 1))
wordCounts = pairs.reduceByKey(lambda x,y: x + y )
wordCounts.pprint(2)

def computeMetrics(rdd):
    global totalLines
    global linesCount
    linesCount=rdd.count()
    totalLines+=linesCount()
    print(rdd.collect())
    print("Lines in RDD :", linesCount, " Total Lines:", totalLines)

lines.foreachRDD(computeMetrics)

def windowMetrics(rdd):
    print("Window RDD size:", rdd.count())
    
windowedRDD = lines.window(6,3)
windowedRDD.foreachRDD(windowMetrics)



    

In [4]:
streamContext.start()

-------------------------------------------
Time: 2020-03-12 02:09:18
-------------------------------------------
('ERM da vinci code and it sucked...', 1)
('da vinci code was a terrible movie.', 1)

Window RDD size: 2
-------------------------------------------
Time: 2020-03-12 02:09:21
-------------------------------------------
('well DA VINCI CODE SUCKS!!', 1)
('I really hate The Da Vinci Code.', 1)
...

Window RDD size: 5
-------------------------------------------
Time: 2020-03-12 02:09:24
-------------------------------------------
('I HATED Da Vinci Code!', 1)
('The Da Vinci code sucks and is also a page turner...', 1)

Window RDD size: 5
-------------------------------------------
Time: 2020-03-12 02:09:27
-------------------------------------------
('I HATED Da Vinci Code!', 1)
('then was the da vinci code', 1)

Window RDD size: 4
-------------------------------------------
Time: 2020-03-12 02:09:30
-------------------------------------------
('I forgot how much I hated Da Vi

Window RDD size: 4
-------------------------------------------
Time: 2020-03-12 02:11:09
-------------------------------------------
("I also think The Da Vinci Code sucked balls and it's the worst piece of shit I've ever read.", 1)
('i loved da vinci code..', 1)
...

Window RDD size: 4
-------------------------------------------
Time: 2020-03-12 02:11:12
-------------------------------------------
('I know Da Vinci Code is going to suck.', 1)

Window RDD size: 4
-------------------------------------------
Time: 2020-03-12 02:11:15
-------------------------------------------
('I think this bolsters my arguments that both England and the Da Vinci Code sucks...', 1)
('The more you say The Da Vinci Code is evil', 1)

Window RDD size: 3
-------------------------------------------
Time: 2020-03-12 02:11:18
-------------------------------------------
('The Da Vinci Code is an awesome book...', 1)
('Da vinci code is an awesome book.', 1)

Window RDD size: 4
-----------------------------------

In [5]:
streamContext.stop()


Window RDD size: 4


In [7]:
print("Overall lines : ", totalLines)

Overall lines :  0


In [2]:
#Spark Machine Learning - Linear Regression
#Problem statement

"""
The input data set contains data about details of various car models. Based on the information provided, the goal 
is to come with a model to predict Miles-per-gallon of a given model.

Techniques Used:

1. Linear Regression ( mulit-variate)
2. Data Imputation - replacing non-numeric data with numeric 
3. Variable Reduction - picking up only relevant features

--------------------------------------
"""
import os
os.chdir("/home/pamba/Desktop/HomeLearning/ApacheSpark/Python")
os.curdir

# Load the csv file into a RDD
autoData = sc.textFile("auto-miles-per-gallon.csv")
autoData.cache()




auto-miles-per-gallon.csv MapPartitionsRDD[4] at textFile at NativeMethodAccessorImpl.java:0

In [3]:
# Remove the first line (contains headers)
dataLines = autoData.filter(lambda x: "CYLINDERS" not in x)
dataLines.count()



398

In [11]:
#Convert the RDD into a Dense Vector. As a part of this exercise
# 1. Remove unwanted columns
# 2. Change non-numeric (values=?) to numeric

import math
from pyspark.mllib.linalg import Vectors

# Use default for average HP
avgHP = sc.broadcast(80.0)

def transformToNumeric(inputStr):
    global avgHP
    attList = inputStr.split(",")
    
    # Replace ? values with a normal value
    hpValue = attList[3]
    if hpValue == "?":
        hpValue = avgHP.value
        
    # Filter out columns not wanted at this stage 
    values = Vectors.dense([float(attList[0]), float(attList[1]), hpValue, float(attList[5]), float(attList[6])])
    
    return values

# Keep only MPG, CYLINDERS, HP, ACCELERATION and MODELYEAR
autoVectors = dataLines.map(transformToNumeric)
autoVectors.collect()

[DenseVector([18.0, 8.0, 130.0, 12.0, 70.0]),
 DenseVector([15.0, 8.0, 165.0, 11.5, 70.0]),
 DenseVector([18.0, 8.0, 150.0, 11.0, 70.0]),
 DenseVector([16.0, 8.0, 150.0, 12.0, 70.0]),
 DenseVector([17.0, 8.0, 140.0, 10.5, 70.0]),
 DenseVector([15.0, 8.0, 198.0, 10.0, 70.0]),
 DenseVector([14.0, 8.0, 220.0, 9.0, 70.0]),
 DenseVector([14.0, 8.0, 215.0, 8.5, 70.0]),
 DenseVector([14.0, 8.0, 225.0, 10.0, 70.0]),
 DenseVector([15.0, 8.0, 190.0, 8.5, 70.0]),
 DenseVector([15.0, 8.0, 170.0, 10.0, 70.0]),
 DenseVector([14.0, 8.0, 160.0, 8.0, 70.0]),
 DenseVector([15.0, 8.0, 150.0, 9.5, 70.0]),
 DenseVector([14.0, 8.0, 225.0, 10.0, 70.0]),
 DenseVector([24.0, 4.0, 95.0, 15.0, 70.0]),
 DenseVector([22.0, 6.0, 95.0, 15.5, 70.0]),
 DenseVector([18.0, 6.0, 97.0, 15.5, 70.0]),
 DenseVector([21.0, 6.0, 85.0, 16.0, 70.0]),
 DenseVector([27.0, 4.0, 88.0, 14.5, 70.0]),
 DenseVector([26.0, 4.0, 46.0, 20.5, 70.0]),
 DenseVector([25.0, 4.0, 87.0, 17.5, 70.0]),
 DenseVector([24.0, 4.0, 90.0, 14.5, 70.0]),
 

In [8]:
# Perform statistical Analysis
from pyspark.mllib.stat import Statistics
autoStats = Statistics.colStats(autoVectors)
autoStats.mean()


array([ 23.51457286,   5.45477387, 104.10050251,  15.56809045,
        76.01005025])

In [9]:
autoStats.variance()


array([  61.08961077,    2.89341544, 1468.09062947,    7.60484823,
         13.67244282])

In [10]:
autoStats.min()


array([ 9.,  3., 46.,  8., 70.])

In [12]:
autoStats.max()


array([ 46.6,   8. , 230. ,  24.8,  82. ])

In [13]:

Statistics.corr(autoVectors)



array([[ 1.        , -0.77539629, -0.77463084,  0.42028891,  0.57926713],
       [-0.77539629,  1.        ,  0.84275215, -0.50541949, -0.3487458 ],
       [-0.77463084,  0.84275215,  1.        , -0.68829885, -0.41559383],
       [ 0.42028891, -0.50541949, -0.68829885,  1.        ,  0.28813695],
       [ 0.57926713, -0.3487458 , -0.41559383,  0.28813695,  1.        ]])

In [36]:
# Transform to a Data Frame for input to Machine Learning
# Drop columns that are not required (low correlation)
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

def transformToLabeledPoint(inStr):
    lp = (float(inStr[0]), Vectors.dense([inStr[1], inStr[2], inStr[4]]))
    return lp

autoLp = autoVectors.map(transformToLabeledPoint)
autoDF = sqlContext.createDataFrame(autoLp, ["label", "features"])
autoDF.select("label", "features").show(10)


+-----+----------------+
|label|        features|
+-----+----------------+
| 18.0|[8.0,130.0,70.0]|
| 15.0|[8.0,165.0,70.0]|
| 18.0|[8.0,150.0,70.0]|
| 16.0|[8.0,150.0,70.0]|
| 17.0|[8.0,140.0,70.0]|
| 15.0|[8.0,198.0,70.0]|
| 14.0|[8.0,220.0,70.0]|
| 14.0|[8.0,215.0,70.0]|
| 14.0|[8.0,225.0,70.0]|
| 15.0|[8.0,190.0,70.0]|
+-----+----------------+
only showing top 10 rows



In [30]:
# Find correlations
numFeatures = autoDF.take(1)[0].features.size


In [32]:
labelRDD = autoDF.rdd.map(lambda lp: float(lp.label))
for i in range(numFeatures):
    featureRDD = autoDF.rdd.map(lambda lp: lp.features[1])
    corr = Statistics.corr(labelRDD, featureRDD, 'pearson')
    print('%d\t%g' % (i, corr))
    
# Split into training and testing data
(trainingData, testData) = autoDF.randomSplit([0.9, 0.1])
trainingData.count()


0	-0.774631
1	-0.774631
2	-0.774631


361

In [23]:
testData.count()

49

In [38]:
# Build the model on training data
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(labelCol='label', featuresCol='features',maxIter=10)
lrModel = lr.fit(trainingData)

print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

IllegalArgumentException: 'requirement failed: Column features must be of type struct<type:tinyint,size:int,indices:array<int>,values:array<double>> but was actually struct<type:tinyint,size:int,indices:array<int>,values:array<double>>.'

In [ ]:


# Predict on the test data
predictions = lrModel.transform(testData)
predictions.select("predictions", "label", "features").show()

from pyspark.ml.evaluation import RegressionEvaluator